# Build a workflow guide

## Building a simple workflow

### Import relevant objects

In [ ]:
from laboneq_applications import workflow

### Define the tasks

In [ ]:
@workflow.task
def measure() -> int:
    return 100


@workflow.task
def analyze(measurement_result: int, threshold: int) -> bool:
    return measurement_result < threshold

### Define the workflow

In this section we go through the process of combining our predefined tasks into a
single workflow.

A workflow in itself should be as simple as possible and it should not contain any complex
operations. Most operations should happen within the tasks.

A workflow can be defined by decorating a Python function with `@workflow` decorator.

#### Important remarks

When a function is marked as a `workflow`, it has some limitations to a normal Python
execution flow:

* Only functions marked as tasks should be called within a workflow definition
* Using Python statements (`if`, `else`, `for`, etc.) should not be used in the Workflow, however they can be used freely in tasks.

The reasons for above limitations is to ensure that a graph of dependencies between tasks
can be created and the `Workflow` can then fully control the execution flow.

#### Workflow references

While the workflow is being constructed, the actual variables (workflow inputs, task outputs) are replaced
with a `Reference` object that then connects the producing and receiving ends of an variable.

By default `Reference` supports only a subset of default Python operations, for example, `__getitem__` and
`__getattr__`. The supported operations can be seen from `Reference` documentation.

### Build a workflow

In [ ]:
@workflow.workflow
def experiment(threshold: int):
    measurement = measure()
    analyze(measurement, threshold)

### Instantiate and run the workflow

In [ ]:
wf = experiment(threshold=101)
wf

In [ ]:
result = wf.run()

### Inspect the results

In [ ]:
result

#### Inspecting the tasks

In [ ]:
result.tasks

There are several ways to get the individual tasks from the `WorkflowResult`

In [ ]:
result.tasks[1], result.tasks["analyze"]

Specific task lookup with indexing

The first argument is the name of the task and the second is an integer or a
`slice`

In [ ]:
result.tasks["analyze", :]  # All tasks named 'analyze'

In [ ]:
result.tasks["analyze", 0]  # First task entry for 'analyze'

Inspecting individual task information

In [ ]:
# Task output
result.tasks["analyze"].output

In [ ]:
# Task input
result.tasks["analyze"].input

### Inspect a workflow that has failed

In case there is an error during the execution of a workflow, we can still inspect the tasks that have run up to the task that triggered the error using `recover()`. Note that `recover()` stores only one execution result and can only be called once; a second call to `recover()` raises an exception.

For experiment workflows, this is useful for debugging a failed compilation task by inspecting the experiment sequence produced by the previous task. 

In this example, we will add an assertion error to the `analyze` task.

In [ ]:
@workflow.task
def measure() -> int:
    return 100


@workflow.task
def analyze(measurement_result: int, threshold: int) -> bool:
    # let's add an error in this task
    if not (measurement_result >= 100 and threshold >= 100):
        raise RuntimeError("Something went wrong.")
    return measurement_result < threshold


@workflow.workflow
def experiment(threshold: int):
    measurement = measure()
    analyze(measurement, threshold)
    workflow.return_("PASS")

In [ ]:
result = experiment(99).run()

In [ ]:
recovered_result = experiment.recover()
recovered_result

In [ ]:
# Check that the measure task returns a result that is >= 100
recovered_result.tasks["measure"].output

In [ ]:
# Check the value of the threshold passed to the taskbook
recovered_result.output

In [ ]:
# Now we know we have to increase the value of the threshold
result = experiment(101).run()
result.output

### Run a partial Workflow

In some cases during development, only parts of a workflow need to be executed.
For this reason, workflows can be 
executed until a specific `task` within the workflow.

In the example below, we will use the previously defined workflow and execute only
the `measure()` part of the workflow to validate everything is working as expected before
continuing to `analyze()` and finishing the workflow.

First we run the experiment workflow with an `until` argument

In [ ]:
exp = experiment(100)
partial_result = exp.run(until="measure")
partial_result.tasks

In [ ]:
# Inspect the partial result
print(partial_result.tasks)
print(partial_result.tasks[0].output)
print(partial_result.output)

After checking that our simple task works as expected, the workflow can be resumed
by calling `.resume()` without the `until` argument.

The already-executed tasks are not executed again, and their results are used in the following
workflow execution.

In [ ]:
result = exp.resume()
print(result.tasks)
print(result.output)